## Exercise 4.8

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data for dataframe via path
path = r'/Users/abrockk12.wv.us/Documents/Career Foundry/Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [3]:
ords_prods_merge.head(100)

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,_merge,price_range,price_range_loc,busiest_day,busiest_days,busiest_time_periods
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Slowest days,Average orders
2,2,473747,1,3,3,12,21.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Slowest days,Most orders
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Slowest day,Slowest days,Average orders
4,4,431534,1,5,4,15,28.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Slowest day,Slowest days,Most orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5895,3226575,360,1,5,12,NaN,196,1,0,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders
96,6137,1469869,377,3,5,17,3.0,196,9,0,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders
97,6259,1927023,387,2,4,10,22.0,196,3,0,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Slowest day,Slowest days,Most orders
98,6740,858092,420,4,1,19,30.0,196,2,0,...,Soda,77,7,9.0,both,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders


In [4]:
ords_prods_merge.shape

(32404859, 21)

In [5]:
# Step 3 Determine the aggregated mean of the 'order_number' column and group by 'department_id'
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### Step 3
After running the groupby on the entire dataframe as oppoosed to the subset, you can see that all departments were included with the entire dataframe. The subset only included a few departments. 

In [6]:
# Step 4 Create a loyalty flag for existing customers using the transform() and loc() functions
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/tmp/ipykernel_24764/3964812562.py:2: FutureWarning: The provided callable <function amax at 0x1129a2d40> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
# Display all rows indicated
pd.options.display.max_rows = None

In [8]:
# loyalty flag 'Loyal customer' for customers with 'max_order' > 40
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

/tmp/ipykernel_24764/4250083166.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [9]:
# loyalty flag 'Regular customer' for customers with 'max_order' 10 < x <= 40
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] >10), 'loyalty_flag'] = 'Regular customer'

In [10]:
# loyalty flag 'New customer' for customers with 'max_order' <= 10
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
ords_prods_merge['user_id']

In [12]:
# Step 5. Compare the spending habits of the 3 types of customers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'sum']})

prices                           
                       mean  min      max          sum
loyalty_flag                                          
Loyal customer    10.386336  1.0  99999.0  106814042.2
New customer      13.294670  1.0  99999.0   83011787.2
Regular customer  12.495717  1.0  99999.0  198391693.2

The average price for the products purchased by the New customer group was the highest at $13.29. Although the loyal customers have purchased often, their average price is the lowest of the 3 groups. 

### High Spender or Low Spender?

In [13]:
# Step 6. Create a spending flag to designate a user as a 'High' or 'Low' spender
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/tmp/ipykernel_24764/1407510991.py:2: FutureWarning: The provided callable <function mean at 0x1129a3a60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [14]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spender'] = 'Low spender'

/tmp/ipykernel_24764/4213123009.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spender'] = 'Low spender'


In [15]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spender'] = 'High spender'

In [16]:
ords_prods_merge.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range,price_range_loc,busiest_day,busiest_days,busiest_time_periods,max_order,loyalty_flag,avg_price,spender
0,0,2539329,1,1,2,8,NaN,196,1,0,...,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,1,2398795,1,2,3,7,15.0,196,1,1,...,both,Mid-range product,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
2,2,473747,1,3,3,12,21.0,196,1,1,...,both,Mid-range product,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender
3,3,2254736,1,4,4,7,29.0,196,1,1,...,both,Mid-range product,Mid-range product,Slowest day,Slowest days,Average orders,10,New customer,6.367797,Low spender
4,4,431534,1,5,4,15,28.0,196,1,1,...,both,Mid-range product,Mid-range product,Slowest day,Slowest days,Most orders,10,New customer,6.367797,Low spender


### Frequent Customers

In [17]:
# Step 7. Determine frequent and non-frequent customers
ords_prods_merge['frequent_user'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/tmp/ipykernel_24764/2049934164.py:2: FutureWarning: The provided callable <function median at 0x112aed800> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['frequent_user'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [18]:
# Flag each user as a 'Frequent', 'Regular', or 'Non-frequent' customer.
ords_prods_merge.loc[ords_prods_merge['frequent_user'] > 20, 'frequency'] = 'Non-frequent customer'

/tmp/ipykernel_24764/3535094988.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['frequent_user'] > 20, 'frequency'] = 'Non-frequent customer'


In [19]:
ords_prods_merge.loc[(ords_prods_merge['frequent_user'] > 10) & (ords_prods_merge['frequent_user'] <= 20), 'frequency'] = 'Regular customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['frequent_user'] <= 10, 'frequency'] = 'Frequent customer'

In [21]:
ords_prods_merge['frequency'].value_counts(dropna=True)

frequency
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: count, dtype: int64

In [22]:
ords_prods_merge.head(60)

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_time_periods,max_order,loyalty_flag,avg_price,spender,frequent_user,frequency
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2,473747,1,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Mid-range product,Slowest day,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,4,431534,1,5,4,15,28.0,196,1,1,...,Mid-range product,Slowest day,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,5,3367565,1,6,2,7,19.0,196,1,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,6,550135,1,7,1,9,20.0,196,1,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,7,3108588,1,8,1,14,14.0,196,2,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,8,2295261,1,9,1,16,0.0,196,4,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,9,2550362,1,10,4,8,30.0,196,1,1,...,Mid-range product,Slowest day,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [23]:
# Export dataframe as a pickle using path
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))